# Code examples

In [1]:
try:
    from main_estimate import *
except:
    print('Error check packages dependencies / Kernel')

## (1) CPIframe

```python
# === Data (index = HICP & Qt without log transf.) ===
df_w = pd.read_excel("data/data_flat.xlsx",sheet_name="weights")
df_q_index = pd.read_excel("data/data_flat.xlsx",sheet_name="QT_index")
df_p_index = pd.read_excel("data/data_flat.xlsx",sheet_name="P_index")

class CPIframe:
    def __init__(self, df_q_index, df_p_index, df_w, country):
        """
        Args:
            df_q_index (DataFrame): The Quantity data (index).
            df_p_index (DataFrame): The Price data (index).
            df_w (DataFrame): The respective Weights data.
            country (str): Location available (EU27,France,Germany,Spain).
        """
````

In [2]:
eu = CPIframe(df_q_index=df_q_index, df_p_index=df_p_index, df_w=df_w, country="EU27")
print([atr for atr in dir(eu) if "__" not in atr])

['country', 'dates', 'flag_sectors', 'framing', 'inflation', 'price', 'price_index', 'qt', 'qt_index', 'sector', 'sector_inf', 'sectors', 'weights']


- *flag_sectors*, *framing* and *sector* are methods for the class *CPIframe*
- *price_index* and *price* (data of *price_index* log-transformed) are subset dataframes of initial *df_p_index* (Price data) restricted to *country*
- *qt_index* and *qt* (data of *qt_index* log-transformed) are subset dataframes of initial *df_q_index* (Quantity data = demand proxy) restricted to *country*

In [3]:
eu.sectors   #COICOP 4-digit categories
#eu.inflation #MoM inflation rate dataframe
#eu.weights    #DataFrame of Annual sector weights in the overall HICP 
#eu.price
#eu.price_index
#eu.qt
#eu.qt_index

{0: 'Bread and cereals',
 1: 'Meat',
 2: 'Fish and seafood',
 3: 'Milk, cheese and eggs',
 4: 'Oils and fats',
 5: 'Fruit',
 6: 'Vegetables',
 7: 'Sugar, jam, honey, chocolate and confectionery',
 8: 'Food products n.e.c.',
 9: 'Coffee, tea and cocoa',
 10: 'Mineral waters, soft drinks, fruit and vegetable juices',
 11: 'Spirits',
 12: 'Wine',
 13: 'Beer',
 14: 'Tobacco',
 15: 'Clothing materials',
 16: 'Garments',
 17: 'Other articles of clothing and clothing accessories',
 18: 'Cleaning, repair and hire of clothing',
 19: 'Shoes and other footwear',
 20: 'Repair and hire of footwear',
 21: 'Actual rentals for housing',
 22: 'Maintenance and repair of the dwelling',
 23: 'Water supply and miscellaneous services relating to the dwelling',
 24: 'Electricity, gas and other fuels',
 25: 'Furniture and furnishings',
 26: 'Carpets and other floor coverings',
 27: 'Repair of furniture, furnishings and floor coverings',
 28: 'Household textiles',
 29: 'Major household appliances whether elect

In [4]:
eu.flag_sectors()

0  : OK  2001-01 ; 2023-09  --  273  obs. --  Bread and cereals
1  : OK  2001-01 ; 2023-09  --  273  obs. --  Meat
2  : OK  2001-01 ; 2023-09  --  273  obs. --  Fish and seafood
3  : OK  2001-01 ; 2023-09  --  273  obs. --  Milk, cheese and eggs
4  : OK  2001-01 ; 2023-09  --  273  obs. --  Oils and fats
5  : OK  2001-01 ; 2023-09  --  273  obs. --  Fruit
6  : OK  2001-01 ; 2023-09  --  273  obs. --  Vegetables
7  : OK  2001-01 ; 2023-09  --  273  obs. --  Sugar, jam, honey, chocolate and confectionery
8  : OK  2001-01 ; 2023-09  --  273  obs. --  Food products n.e.c.
9  : OK  2001-01 ; 2023-09  --  273  obs. --  Coffee, tea and cocoa
10  : OK  2001-01 ; 2023-09  --  273  obs. --  Mineral waters, soft drinks, fruit and vegetable juices
11  : OK  2001-01 ; 2023-09  --  273  obs. --  Spirits
12  : OK  2001-01 ; 2023-09  --  273  obs. --  Wine
13  : OK  2001-01 ; 2023-09  --  273  obs. --  Beer
14  : OK  2001-01 ; 2023-09  --  273  obs. --  Tobacco
15  : end missing - 2001-01 ; 2022-04  -

## (2) sector_estimation

Builds on a `CPIframe`object.

```python
class sector_estimation:
    def __init__(self,
                 meta:CPIframe,
                 col:int,
                 order:Union[int, str]="auto",maxlag=24,trend="n",
                 shapiro:bool=True,
                 shapiro_robust:bool=False,
                 sheremirov:bool=True,
                 sheremirov_window:list=[1,11]):
        """
        Args:
            - `meta`: `CPIframe` object
            - `col`: sector column number in [0,93]
            - If too litte data for sector 'col' then raises ValueError
            - VAR model is built with first diff then demeand log-transformed data
            - `classify_inflation`: if False only returns Shapiro and Sheremirov classification in binary form 
                Otherwise returns [classified weighted inflation rate] : 1(dem)*weight*inf_rate & 1(sup)*w*inf_rate 
                
            `VAR parametrization`
            order: if "auto" VAR order is automatically selected. Else requires an integer
            maxlag: higher bound of order selection
            trend: should remain "n" because data have been demeaned and are supposed to be stationary (no trend)
            
            `Labeling methods`
            shapiro: if True computes baseline Shapiro(2022) labeling method with reduced-form estimated VAR
            shapiro_robustness: if True also computes alternative labeling methodologies
            sheremirov: if True computes baseline Sheremirov(2022) labeling method
            sheremirov_window: [Transitory,Persistent] parametrization of step classification algo step5
        """

**ex: For a given sector in column col=64**

#### 1/ Automatic lag order selection

In [5]:
# When order is automatically selected results accessible via aic and bic are accessed via (...).aic or (...).bic
test_sect_estim1 = sector_estimation(meta=eu, col=64, shapiro_robust=True)
test_sect_estim12 = sector_estimation(meta=eu, col=64, shapiro_robust=True, classify_inflation=False)
test_sect_estim1.sec_name

'Recording media'

In [16]:
#test_sect_estim1.aic.summary() #Estimated model with AIC
#test_sect_estim1.aic.shapiro #Shapiro classification (without alternative methods)
print('> Shapiro classification')
print('Baseline:dem-sup \nSmoothed classification:dem-sup_j1 to dem-sup_j3 \nParametric classification:dem-sup_param')
display(test_sect_estim1.aic.shapiro_robust)
print('> Sheremirov classification')
display(test_sect_estim1.sheremirov)

> Shapiro classification
Baseline:dem-sup 
Smoothed classification:dem-sup_j1 to dem-sup_j3 
Parametric classification:dem-sup_param


,dem,sup,dem_j1,sup_j1,dem_j2,sup_j2,dem_j3,sup_j3,dem_param,sup_param
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-05,-0.045945,-0.0,-0.045945,-0.0,-0.0,-0.045945,-0.0,-0.045945,-0.0342,-0.011745
2023-06,0.0,0.040719,0.040719,0.0,0.040719,0.0,0.040719,0.0,0.018098,0.022621
2023-07,-0.0,-0.019474,-0.019474,-0.0,-0.019474,-0.0,-0.0,-0.019474,-0.009415,-0.010059
2023-08,0.0,0.018306,0.0,0.018306,0.0,0.018306,0.018306,0.0,0.003604,0.014702


> Sheremirov classification


,dem,sup,dem_pers,dem_trans,sup_pers,sup_trans
2000-01,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-05,-0.0,-0.045945,-0.0,-0.0,-0.045945,-0.0
2023-06,0.0,0.040719,0.0,0.0,0.040719,0.0
2023-07,-0.0,-0.019474,-0.0,-0.0,-0.019474,-0.0
2023-08,0.0,0.018306,0.0,0.0,0.018306,0.0


In [17]:
print('> Shapiro classification classify_inflation=False')
print('Baseline:dem-sup \nSmoothed classification:dem-sup_j1 to dem-sup_j3 \nParametric classification:dem-sup_param')
display(test_sect_estim12.aic.shapiro_robust)
print('> Sheremirov classification classify_inflation=False')
display(test_sect_estim12.sheremirov)

> Shapiro classification classify_inflation=False
Baseline:dem-sup 
Smoothed classification:dem-sup_j1 to dem-sup_j3 
Parametric classification:dem-sup_param


,dem,sup,dem_j1,sup_j1,dem_j2,sup_j2,dem_j3,sup_j3,dem_param,sup_param
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-05,1,0,1,0,0,1,0,1,0.744368,0.255632
2023-06,0,1,1,0,1,0,1,0,0.444465,0.555535
2023-07,0,1,1,0,1,0,0,1,0.483442,0.516558
2023-08,0,1,0,1,0,1,1,0,0.196894,0.803106


> Sheremirov classification classify_inflation=False


,dem,sup,dem_pers,dem_trans,sup_pers,sup_trans
2000-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-04,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...
2023-05,0,1,0,0,1,0
2023-06,0,1,0,0,1,0
2023-07,0,1,0,0,1,0
2023-08,0,1,0,0,1,0


In [7]:
test_sect_estim1.inflation
# inflation is MoM%HICP

,infw
2000-01,NaN
2000-02,NaN
2000-03,NaN
2000-04,NaN
2000-05,NaN
...,...
2023-05,-0.045945
2023-06,0.040719
2023-07,-0.019474
2023-08,0.018306


#### 2/ Fixed order

In [8]:
# When order is hand-fixed results are accessible via (...).estimate
test_sect_estim2 = sector_estimation(meta=eu, col=64, order=12, shapiro_robust=True)
test_sect_estim22 = sector_estimation(meta=eu, col=64, order=12, shapiro_robust=True, classify_inflation=False)

In [11]:
#test_sect_estim2.estimate.summary() #Estimated model with selected order
#test_sect_estim2.estimate.shapiro #(without alternative methods)
print('> Shapiro classification')
display(test_sect_estim2.estimate.shapiro_robust)
print('> Sheremirov classification')
display(test_sect_estim2.sheremirov)

> Shapiro classification


,dem,sup,dem_j1,sup_j1,dem_j2,sup_j2,dem_j3,sup_j3,dem_param,sup_param
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-05,-0.045945,-0.0,-0.0,-0.045945,-0.0,-0.045945,-0.0,-0.045945,-0.024098,-0.021847
2023-06,0.0,0.040719,0.040719,0.0,0.040719,0.0,0.0,0.040719,0.0107,0.030019
2023-07,-0.0,-0.019474,-0.0,-0.019474,-0.019474,-0.0,-0.019474,-0.0,-0.009729,-0.009745
2023-08,0.0,0.018306,0.0,0.018306,0.0,0.018306,0.018306,0.0,0.008376,0.00993


> Sheremirov classification


,dem,sup,dem_pers,dem_trans,sup_pers,sup_trans
2000-01,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-05,-0.0,-0.045945,-0.0,-0.0,-0.045945,-0.0
2023-06,0.0,0.040719,0.0,0.0,0.040719,0.0
2023-07,-0.0,-0.019474,-0.0,-0.0,-0.019474,-0.0
2023-08,0.0,0.018306,0.0,0.0,0.018306,0.0


## (3) CPIlabel

Builds on a `CPIframe`object.

Iterates over all sectors.

```python
class CPIlabel:
    def __init__(self,meta:CPIframe,
                 order:Union[int, str]="auto",maxlag=24,
                 shapiro_robust:bool=False,
                 sheremirov_window:list[int,int]=[1,11]):
        """
        Args:
            - `meta`: `CPIframe` object
            - `order`: if "auto" VAR order is automatically selected. Else requires an integer
            - `maxlag`: higher bound of order selection (if order="auto")
            - `shapiro_robustness`: if True also computes alternative labeling methodologies
            - `sheremirov_window: [Transitory,Persistent] parametrization of step classification algo step5 
            -  NB1: VAR models are built with first diff then demeand log-transformed data
        """
````